In [ ]:

# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import tensorflow as tf
import tensorflow_datasets as tfds

import logging
import random

import tensorflow.keras as keras
from CompressionLibrary.utils import calculate_model_weights
from CompressionLibrary.custom_layers import DenseSVD, MLPConv
from CompressionLibrary.reward_functions import reward_MnasNet as calculate_reward
from uuid import uuid4
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from CompressionLibrary.CompressionTechniques import InsertDenseSVD, MLPCompression
import copy

import gc

from deap import base, creator
from deap import algorithms
import random
from deap import tools


current_os = 'windows'


dataset_name = 'imagenet2012'
batch_size = 32

agent_name = 'scatter_search' + '_' + dataset_name


if current_os == 'windows':
    data_path = f'G:\\Python projects\\ModelCompressionRL\\data\\'
    log_path = f'G:\\Python projects\\ModelCompressionRL\\data\\logs\\ModelCompression_{agent_name}.log'
    exploration_filename = data_path + f'stats/{agent_name}_training.csv'
    test_filename = data_path + f'stats\\{agent_name}_testing.csv'
    figures_path = data_path+f'figures\\{agent_name}'
    datasets_path = "G:\\ImageNet 2012\\"#data_path+"datasets"
else:
    data_path = './data'
    log_path = f'/home/A00806415/DCC/ModelCompression/data/logs/ModelCompression_{agent_name}.log'
    exploration_filename = data_path + f'/stats/{agent_name}_training.csv'
    test_filename = data_path + f'/stats/{agent_name}_testing.csv'
    figures_path = data_path+f'/figures/{agent_name}'


# logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)
logging.basicConfig(level=logging.DEBUG, handlers=[
    logging.FileHandler(log_path, 'w+')],
    format='%(asctime)s -%(levelname)s - %(funcName)s -  %(message)s')
logging.root.setLevel(logging.DEBUG)

log = logging.getLogger('tensorflow')
log.setLevel(logging.ERROR)

logger = logging.getLogger(__name__)


# layer_name_list = ['conv2d_1',  'dense', 'dense_1']
layer_name_list = [ 'block2_conv1', 'block2_conv2', 
                    'block3_conv1', 'block3_conv2', 'block3_conv3',
                    'block4_conv1', 'block4_conv2', 'block4_conv3',
                    'block5_conv1', 'block5_conv2', 'block5_conv3',
                    'fc1', 'fc2']


optimizer = tf.keras.optimizers.Adam(1e-5)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
def create_model(dataset_name, train_ds, valid_ds):
    
    train_metric = tf.keras.metrics.SparseCategoricalAccuracy()


    model = tf.keras.applications.vgg16.VGG16(
                            include_top=True,
                            weights='imagenet',
                            input_shape=(224,224,3),
                            classes=1000,
                            classifier_activation='softmax'
                        )

    model.compile(optimizer=optimizer, loss=loss_object,
                    metrics=train_metric)

    return model    

@tf.function
def imagenet_preprocessing(img, label):
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, size=(224,224), method='bicubic')
    img = tf.keras.applications.vgg16.preprocess_input(img, data_format=None)
    return img, label

def load_dataset(dataset, batch_size=64):
    splits, info = tfds.load('imagenet2012', as_supervised=True, with_info=True, shuffle_files=True, 
                                split=['validation', 'validation','validation'], data_dir=datasets_path)

    #   splits, info = tfds.load('imagenet2012', as_supervised=True, with_info=True, shuffle_files=True, 
    #                               split=['train[:80%]', 'train[80%:]','validation'], data_dir=data_path)
                                
    (_, validation_examples, _) = splits
    num_examples = info.splits['validation'].num_examples#info.splits['train'].num_examples

    num_classes = info.features['label'].num_classes
    input_shape = info.features['image'].shape

    input_shape = (224,224,3)
    valid_ds = validation_examples.map(imagenet_preprocessing, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return valid_ds, input_shape, num_classes

def get_max_hidden_units(model, layer_list):
    max_values = []
    for layer_name in layer_list:
        layer = model.get_layer(layer_name)
        if isinstance(layer, tf.keras.layers.Conv2D):
            kernel, bias = layer.get_weights()
            h, w, c, filters = kernel.shape
            weights = tf.reshape(kernel, shape=[-1, filters])
            input_size, _ = weights.shape
            units = filters
            
        elif isinstance(layer, tf.keras.layers.Dense):
            weights, bias = layer.get_weights()
            input_size , units = weights.shape
            
        max_hidden_units = (input_size * units)//(input_size+units)
        max_values.append(max_hidden_units)

    return max_values



valid_ds, input_shape, _ = load_dataset(dataset_name, batch_size)
train_ds = test_ds = valid_ds
parameters = {}
parameters['InsertDenseSVD'] = {'layer_name': None, 'percentage': None, 'hidden_units':None}
parameters['MLPCompression'] = {'layer_name': None, 'percentage': None, 'hidden_units':None}


optimizer = tf.keras.optimizers.Adam(1e-5)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
train_metric = tf.keras.metrics.SparseCategoricalAccuracy()
verbose = False

# Create a the original model to calculate stats before.
temp_model = create_model(dataset_name=dataset_name, train_ds=train_ds, valid_ds=valid_ds)
weights_before = calculate_model_weights(temp_model)
test_loss, test_acc_before = temp_model.evaluate(test_ds, verbose=verbose)
val_loss, val_acc_before = temp_model.evaluate(valid_ds, verbose=verbose)


max_hidden_units = get_max_hidden_units(temp_model, layer_name_list)
del temp_model

logging.info(f'Max hidden values are {max_hidden_units}.')

logger.info(f'Max number of singular values per layer are : {max_hidden_units}.')

eval_dict = dict()
def evaluation_function(ind):
    ind = fix_solution(ind)
    callbacks = []
    model = create_model(dataset_name=dataset_name, train_ds=train_ds, valid_ds=valid_ds)
    inputs = tf.keras.layers.Input(shape=input_shape)
    if isinstance(model.layers[0], tf.keras.layers.InputLayer):
        x = model.layers[1](inputs)
        start = 2
    else:
        x = model.layers[0](inputs)
        start = 1
    for layer in model.layers[start:]:
        if layer.name in layer_name_list:
            weights, bias = layer.get_weights()
            hidden_units = ind[layer_name_list.index(layer.name)]

            if isinstance(layer, tf.keras.layers.Conv2D):
                config = layer.get_config()
                filters = config['filters']
                kernel_size = config['kernel_size']
                activation = config['activation']
                padding = config['padding']
                weights = tf.reshape(weights, shape=[-1, filters])
                
                try:
                    s, u, v = tf.linalg.svd(weights, full_matrices=False)
                except Exception as e:
                    print(e)
                    with tf.device('/CPU:0'):
                        s, u, v = tf.linalg.svd(weights, full_matrices=False)

                u = tf.slice(u, begin=[0, 0], size=[weights.shape[0], hidden_units])
                s = tf.slice(s, begin=[0], size=[hidden_units])

                # Transpose V as V is returned as V^T.
                v = tf.slice(tf.linalg.matrix_transpose(v), begin=[0, 0], size=[hidden_units, filters])
                n = tf.matmul(tf.linalg.diag(s), v)

                new_layer = MLPConv(filters=filters, hidden_units=hidden_units, kernel_size=kernel_size, padding=padding,
                  activation=activation,
                  name=layer.name + '/MLPConv')
                
                new_layer(layer.input)

                new_layer.set_weights([u, n, bias])

            elif isinstance(layer, tf.keras.layers.Dense):
                _ , units = weights.shape
                activation = layer.get_config()['activation']
                try:
                    s, u, v = tf.linalg.svd(weights, full_matrices=False)
                except Exception as e:
                    print(e)
                    with tf.device('/CPU:0'):
                        s, u, v = tf.linalg.svd(weights, full_matrices=False)

                u = tf.slice(u, begin=[0, 0], size=[weights.shape[0], hidden_units])
                s = tf.slice(s, begin=[0], size=[hidden_units])

                # Transpose V as V is returned as V^T.
                v = tf.slice(tf.linalg.matrix_transpose(v), begin=[0, 0], size=[hidden_units, units])
                n = tf.matmul(tf.linalg.diag(s), v)
                new_layer = DenseSVD(units=units, hidden_units=hidden_units,
                    activation=activation,
                    name=layer.name + '/DenseSVD')
                new_layer(layer.input)

                new_layer.set_weights([u.numpy(), n, bias])
                            
            else:
                raise f'Layer {layer.name} has an unsupported layer type for compression.'
            
            x = new_layer(x)
        else:
            x = layer(x)
       
    model = tf.keras.Model(inputs, x)
    
    model.compile(optimizer, loss_object)
    seq_key = ','.join(map(str,ind))
    if seq_key not in eval_dict.keys():

        logger.debug('Evaluating model because it has not been explored before.')
        start = datetime.now()
        test_loss, test_acc_after = model.evaluate(test_ds, verbose=verbose)
        val_loss, val_acc_after = test_loss, test_acc_after #model.evaluate(valid_ds, verbose=verbose)
        end = datetime.now()
        logger.debug(f'Evaluation took {(end-start).total_seconds():2f} seconds. ')
    
    else:
        test_acc_after, weights_after = eval_dict[seq_key]
        logger.debug(f'Found evaluation of {ind}: ({test_acc_after}, {weights_after})')
    weights_after = calculate_model_weights(model)
    stats = {
                'weights_before': weights_before, 
                'weights_after': weights_after, 
                'accuracy_after': test_acc_after, 
                'accuracy_before': test_acc_before}

    reward = calculate_reward(stats)

    eval_dict[seq_key] = (stats['accuracy_after'], stats['weights_after'])

    logger.debug(f'Solution {ind} has {weights_after} weights and an accuracy of {test_acc_after}. Reward is {reward}')
    del model
    gc.collect()
    return stats['accuracy_after'], stats['weights_after']


def mutation(ind, indpb, max_delta=10):
    for action_idx in range(len(ind)):
        if random.uniform(0.0, 1.0) < indpb:
            delta = np.random.randint(low=-max_delta, high=max_delta)
            ind[action_idx] = np.clip(ind[action_idx] + delta, a_min=1, a_max=max_hidden_units[action_idx]+1)
    return ind,


def fix_solution(ind):
    for action_idx in range(len(ind)):
        ind[action_idx] = np.clip(ind[action_idx], 1, max_hidden_units[action_idx]+1)
    return ind


def div_gen_method(n, max_hidden_units):
    n_variables = len(max_hidden_units)
    subranges = 4
    freq = np.ones(shape=(n_variables, subranges))
    
    population = []
    for _ in range(n):
        new_sol = []
        for j, max_value in enumerate(max_hidden_units):
            # Add one to consider no compression.
            subrange_size = (max_value+1) //4
            p = freq[j]/np.sum(freq[j])
            chosen_subrange = np.random.choice(subranges,size=1, replace=False, p=p)[0]
            start = subrange_size * chosen_subrange
            end = subrange_size * (chosen_subrange+1)
            new_sol.append(random.randint(start, end))
            
            freq[j, chosen_subrange] += 1
        population.append(new_sol)
        print(new_sol)
    return population




div_gen_method(10, max_hidden_units)

        








In [ ]:
# Size of the refSet.
b = 20
# Initial number of high-quality solutions in the refSet.
b1 = 10
# Initial number of diverse solutions in refSet.
b2 = 10



class Solution:
    def __init__(self, x, evaluations=None) -> None:
        self.x = x
        self.evaluations = evaluations


class ReferenceSet:
    def __init__(self, size=0) -> None:
        self.solutions = []
        self.size = size
        self.newSol = False
        

    

def div_gen_method(n, max_hidden_units):
    n_variables = len(max_hidden_units)
    subranges = 4
    freq = np.ones(shape=(n_variables, subranges))
    
    population = []
    for _ in range(n):
        new_sol = []
        for j, max_value in enumerate(max_hidden_units):
            # Add one to consider no compression.
            subrange_size = (max_value+1) //4
            p = (1 + np.max(freq[j]) - freq[j])    
            p = p / np.sum(p)
            chosen_subrange = np.random.choice(subranges,size=1, replace=False, p=p)[0]
            start = subrange_size * chosen_subrange
            end = subrange_size * (chosen_subrange+1)
            new_sol.append(random.randint(start, end))
            
            freq[j, chosen_subrange] += 1
        population.append(new_sol)

    return population



def impove_method(sol):


    return sol


def update_reference_set_method(refset):
    pass


def subset_generation_method(refset):
    pass

def solution_combination_method(sol1, sol2):
    pass

